# Example 4: Solving Kepler’s Equation and Verifying Eccentric Anomaly


### Problem Statement

This example demonstrates how to numerically solve **Kepler’s Equation** to determine the eccentric anomaly $E$ from the mean anomaly $M_e$ and orbital eccentricity $e$.

**Kepler’s Equation:**

$$
M_e = E - e \sin E
$$

Because the equation is transcendental, there is no closed-form analytical solution. Instead, we solve it using iterative root-finding methods.

### Objectives

1. Implement a numerical root-finding method (SciPy’s `fsolve`) to compute $E$.
2. Validate the solver against textbook/class examples.
3. Extend the computation to verify:
   - Time since perigee
   - True anomaly $\theta$
   - Internal consistency of orbital mechanics functions in `orbit_util.py`


### Verification Example (from lecture notes)

- Eccentricity: $e = 0.3725$
- Mean anomaly: $M_e = 1.36 \; \text{rad}$
- Expected eccentric anomaly: $E = 1.7281 \; \text{rad}$


### Method

We use the class `Orbit_2body` (from orbit_util.py) to:

- Compute specific angular momentum $h$.
- Propagate time since perigee for a given true anomaly.
- Numerically solve Kepler’s equation using `fsolve`.
- Compare results with expected textbook values.

This confirms the correctness of the Kepler solver and the orbital mechanics pipeline.


### Code

Here is the example code:

In [ ]:
import numpy as np
from orbit_util import Orbit_2body
from scipy.optimize import fsolve

# -------------------------
# Helper functions
# -------------------------
def rad2deg(rad):
    """Convert radians to degrees."""
    return np.rad2deg(rad)

def deg2rad(deg):
    """Convert degrees to radians."""
    return np.deg2rad(deg)

def compare(val1, val2, tol=1e-2):
    """Compare two values with tolerance."""
    return abs(val1 - val2) <= tol


In [ ]:
# -------------------------
# Initialize orbit solver
# -------------------------
orbit = Orbit_2body()

def validate_shadow_example(apogee_toward_sun=True):
    """
    Validate shadow duration calculation using orbital geometry.

    Parameters
    ----------
    apogee_toward_sun : bool
        If True → apogee points toward the Sun.
        If False → perigee points toward the Sun.
    """

    print("\nSlide 27–35: Shadow duration calculation")

    # Earth radius [km]
    RE = 6378

    # Perigee and apogee distances
    rp = RE + 500     # Perigee radius [km]
    ra = RE + 5000    # Apogee radius [km]

    # Semi-major axis and eccentricity
    a = (rp + ra) / 2
    e = (ra - rp) / (ra + rp)
    mu = orbit.mu  # Earth's gravitational parameter

    # Compute specific angular momentum at perigee
    r_vec = np.array([rp, 0, 0])
    v_vec = np.array([0, np.sqrt(mu * (1 + e) / rp), 0])
    _, h = orbit.specific_angular_momentum(r_vec, v_vec)

    # Orbital radius as function of true anomaly θ
    def r_theta(theta):
        return (h**2 / mu) / (1 + e * np.cos(theta))

    # Condition for shadow entry/exit
    def f(theta):
        return np.sin(theta) - RE / r_theta(theta)

    # Solve for θ intersections
    theta1 = fsolve(f, deg2rad(30))[0]
    theta2 = -theta1 if apogee_toward_sun else np.pi - theta1

    # Compute times since perigee
    t1, _ = orbit.time_since_perigee(theta1, h=h, e=e)
    t2, _ = orbit.time_since_perigee(theta2, h=h, e=e)
    shadow_time = abs(t2 - t1)

    # Display results
    if apogee_toward_sun:
        print("Configuration: Apogee toward Sun")
    else:
        print("Configuration: Perigee toward Sun")

    print(f"Theta entry: {rad2deg(theta1):.2f}°, exit: {rad2deg(theta2):.2f}°")
    print(f"Shadow duration: {shadow_time:.2f} s = {shadow_time/60:.2f} min")


In [ ]:
# -------------------------
# Case 1: Apogee toward Sun
# -------------------------
validate_shadow_example(apogee_toward_sun=True)


In [ ]:
# -------------------------
# Case 2: Perigee toward Sun
# -------------------------
validate_shadow_example(apogee_toward_sun=False)
